In [861]:
# !pip install beautifulsoup4

In [279]:
import pandas as pd
import unicodedata
import os
import io
import requests
import selenium
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
import datetime as dt
import time
from random import randint
from time import sleep

In [55]:
# import the contents of CSPP_breakdown_by_sector_rating_country excel sheets into seperate dataframes
q1_2021_df = pd.read_excel (r'/Users/gabbyvinco/Desktop/CSPP_breakdown_by_sector_rating_country(1).xlsx', sheet_name='CSPP_breakdown_Q1_2021')
q3_2020_df = pd.read_excel (r'/Users/gabbyvinco/Desktop/CSPP_breakdown_by_sector_rating_country(1).xlsx', sheet_name='CSPP_breakdown_Q3_2020')
q1_2020_df = pd.read_excel (r'/Users/gabbyvinco/Desktop/CSPP_breakdown_by_sector_rating_country(1).xlsx', sheet_name='CSPP_breakdown_Q1_2020')
q3_2019_df = pd.read_excel (r'/Users/gabbyvinco/Desktop/CSPP_breakdown_by_sector_rating_country(1).xlsx', sheet_name='CSPP_breakdown_Q3_2019')
q1_2019_df = pd.read_excel (r'/Users/gabbyvinco/Desktop/CSPP_breakdown_by_sector_rating_country(1).xlsx', sheet_name='CSPP_breakdown_Q1_2019')
q3_2018_df = pd.read_excel (r'/Users/gabbyvinco/Desktop/CSPP_breakdown_by_sector_rating_country(1).xlsx', sheet_name='CSPP_breakdown_Q3_2018')
q1_2018_df = pd.read_excel (r'/Users/gabbyvinco/Desktop/CSPP_breakdown_by_sector_rating_country(1).xlsx', sheet_name='CSPP_breakdown_Q1_2018')
q3_2017_df = pd.read_excel (r'/Users/gabbyvinco/Desktop/CSPP_breakdown_by_sector_rating_country(1).xlsx', sheet_name='CSPP_breakdown_Q3_2017')


In [56]:
# import the contents of CSPP_PEPP_corporate_bond_holdings_20210430 into a dataframe
corporate_bond_holdings_df = pd.read_excel (r'/Users/gabbyvinco/Desktop/CSPP_PEPP_corporate_bond_holdings_20210430.xlsx')

In [377]:
# remove accents from companies in ISSUER column
corporate_bond_holdings_df['ISSUER'] = corporate_bond_holdings_df['ISSUER'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
# corporate_bond_holdings_df

In [383]:
# create a list of unique ISSUERS to run through the API
unique_issuers = corporate_bond_holdings_df["ISSUER"].unique()
# unique_issuers

In [238]:
organizations_df = pd.DataFrame(unique_issuers)
# organizations_df

In [60]:
# save the df to csv file
# organizations_df.to_csv (r'/Users/gabbyvinco/Desktop/unique_companies.csv', index = False, header=True)

In [210]:
# unique_issuers

In [231]:
orgs_without_abbrv = []

In [232]:
# trying to create some rules to fix the issues with company names

# drop the acronym on the end of the word
for item in unique_issuers:
    item = item.split()
    if len(item) == 2:
        word_count = len(item) 
        word = item[0]
        orgs_without_abbrv.append(word)
    if len(item) >= 3:
        last_word = len(item) - 1
        drop_abbrv = item[:last_word]
        drop_abbrv = " ".join(drop_abbrv)
#         print(drop_abbrv)
        if ", " in drop_abbrv:
            no_comma = drop_abbrv.split(", ")
            no_comma_str = no_comma[0]
#             print(no_comma_str)
            orgs_without_abbrv.append(no_comma_str)
        else:
            orgs_without_abbrv.append(drop_abbrv)
    


In [234]:
# orgs_without_abbrv

# Using Selenium to scrape Google Search to correct names for API search

In [833]:
# unique_issuers

In [595]:
first_word_unique_issuers = []
unique_issuers_count = []

In [596]:
for unique in unique_issuers:
    unique = unique.split()
    length = len(unique)
    unique_issuers_count.append(length)
    unique = unique [0]
    first_word_unique_issuers.append(unique)

In [834]:
# first_word_unique_issuers
# unique_issuers_count

In [842]:
# Install and initiate driver
driver = webdriver.Chrome(ChromeDriverManager().install())

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 90.0.4430
[WDM] - Get LATEST driver version for 90.0.4430


[WDM] - Driver [/Users/gabbyvinco/.wdm/drivers/chromedriver/mac64/90.0.4430.24/chromedriver] found in cache


In [843]:
# URL to test the individual sneaker page functions

cbonds_url = 'https://cbonds.com'
google_url = 'https://www.google.com/search?q=Holding+de+Infrastructures+de+Transport+S.A.S.&oq=Holding+de+Infrastructures+de+Transport+S.A.S.&aqs=chrome..69i57j69i60.935j0j15&sourceid=chrome&ie=UTF-8'
sec_page = 'https://sec.report/CIK/0000313216'
my_url = sec_page

# open and maximize the browser window
driver.get(my_url)
# driver.maximize_window()

In [844]:
# initiate the action object
action = webdriver.ActionChains(driver)

In [845]:
# create a wait with a timeout of 5 seconds
wait = WebDriverWait(driver, 5)

In [562]:
# bypass agreement
time.sleep(3)

button = driver.find_element_by_xpath('/html/body/div[3]/div[3]/span/div/div/div[3]/button[2]')
button.click()

In [569]:
goog_corrected_names_list = []
sidebar_info = []

In [570]:
def google_name_correction(list):
    for item in list:
        
        goog_search_bar = driver.find_element_by_name("q")
        
        # small break
        sleep(randint(1,2))
        
        # click search bar
        goog_search_bar.click()
        
        # small break
        sleep(randint(1,3))
        
        # select items in search bar
        goog_search_bar.send_keys(Keys.COMMAND, 'a')
        
        # small break
        sleep(randint(1,2))
        
        # clear search bar
        goog_search_bar.send_keys(Keys.BACKSPACE)

        # small break
        sleep(randint(1,2))
        
        # type in new item into search bar
        goog_search_bar.send_keys(item)
        
        # small break
        sleep(randint(1,2))
        
        # hit enter and perform search
        goog_search_bar.send_keys(Keys.ENTER)
        
        # break 
        time.sleep(3)
        
        # grab entry from page
        google_first_entry = driver.find_element_by_class_name('LC20lb')
        
        goog_text = google_first_entry.text
#         print(goog_text)
        goog_corrected_names_list.append(goog_text)
        
        try:
            goog_side_info = driver.find_element_by_xpath('/html/body/div[7]/div/div[9]/div[2]/div/div/div[2]/div[1]/div/div[1]/div/div/div[2]/h2')
            goog_extra_info = goog_side_info.text
#             print(goog_extra_info)
            sidebar_info.append(goog_extra_info)
        except NoSuchElementException:
            no_info = "no_side_info"
            sidebar_info.append(no_info)
            continue
            
        time.sleep(5)
            


In [ ]:
google_name_correction(unique_issuers)

In [576]:
# create another variable just incase I overwrite the list
goog_unique = sidebar_info

In [ ]:
# exit from the driver
driver.quit()

In [581]:
# create a list to add the split info into
company_name = []

In [582]:
# function to split the items in goog_unique list
for t in goog_unique:
    title = t.split(' (')
    title = title[0]
    company_name.append(title)

In [1]:
# company_name

In [778]:
# create another variable just incase I overwrite the list
# create a count for the loop below
goog_search_result = goog_corrected_names_list
count = 0

In [779]:
# create a new list to add items to below
new_list = []

In [780]:
# search for items by matching
for script in goog_search_result:
    corresponding_name = first_word_unique_issuers[count]
    find_word = script[script.find(corresponding_name):]
    new_list.append(find_word)
    count += 1


In [782]:
# create new list to add split items to below
# create a count for loop below
drop_unneccesary = []
new_count = 0

In [783]:
# loop through new_list and remove the issues with only one letter
for item in new_list:
    item = item.split()
#     print(item)
    if len(item) == 1:
        item = "none"
        drop_unneccesary.append(item)
        new_count += 1
        pass
    else:
        number = unique_issuers_count[new_count]
#         print(number)
        selected_item = item[:number]
        selected_item = " ".join(selected_item)
        drop_unneccesary.append(selected_item)
        new_count += 1
        

In [784]:
# drop_unneccesary

In [785]:
# creating lists for the loops to clean the data below
dropped_line = []
dropped_colon = []
dropped_hyphen = []
remove_commas = []
remove_com = []
remove_dot_info = []
remove_homepage = []
remove_ellipses = []
remove_strange_symbol = []
remove_2020 = []

In [786]:
# removes everything after | from the google results 
for item in drop_unneccesary:
    item = item.split("|")
    item = item[0]
    dropped_line.append(item)

In [788]:
# removes everything after : from the google results 
for item in dropped_line:
    item = item.split(":")
    item = item[0]
    dropped_colon.append(item)

In [790]:
# removes everything after - from the google results 
for item in dropped_colon:
    item = item.split("-")
    item = item[0]
    dropped_hyphen.append(item)

In [793]:
# removes everything after , from the google results
for item in dropped_hyphen:
    item = item.split(", ")
    item = item[0]
    remove_commas.append(item)

In [795]:
# removes everything after .com from the google results
for item in remove_commas:
    item = item.split(".com")
    item = item[0]
    remove_com.append(item)

In [797]:
# removes everything after . Info from the google results
for item in remove_com:
    item = item.split(". Info")
    item = item[0]
    remove_dot_info.append(item)

In [799]:
# removes everything after Homepage from the google results
for item in remove_dot_info:
    item = item.split("Homepage")
    item = item[0]
    remove_homepage.append(item)

In [801]:
# removes everything after ... from the google results
for item in remove_homepage:
    item = item.split("...")
    item = item[0]
    remove_ellipses.append(item)

In [803]:
# removes everything after – from the google results
for item in remove_ellipses:
    item = item.split("–")
    item = item[0]
    remove_strange_symbol.append(item)

In [805]:
# removes everything after Report 2 from the google results
for item in remove_strange_symbol:
    item = item.split("Report 2")
    item = item[0]
    remove_2020.append(item)

In [807]:
# create new list and count for loop below to replace none values
compound_list = []
name_count = 0

In [808]:
# loop to replace none values with items from company_name
for item in remove_2020:
    if item == "none":
        side_bar_name = company_name[name_count]
        compound_list.append(side_bar_name)
        name_count += 1
    else:
        compound_list.append(item)
        name_count += 1

In [813]:
# compound_list

In [810]:
# create new list and restore count to zero
complete_list = []
name_count = 0

In [811]:
# loop to replace the items with value no_side_info with the values from unique_issuers
for item in compound_list:
    if item == "no_side_info":
        og_name = unique_issuers[name_count]
        complete_list.append(og_name)
        name_count += 1
    else:
        complete_list.append(item)
        name_count += 1

In [1]:
# complete_list

In [826]:
# check and see how many companies we have in the list
len(complete_list)

360

In [829]:
# add list to dataframe so it can be saved
complete_list = pd.DataFrame(complete_list)

In [830]:
# save the complete list to csv file
# complete_list.to_csv (r'/Users/gabbyvinco/Desktop/google_list.csv', index = False, header=True)

# Selenium Scraping of Cbonds.com

In [ ]:
# make list of ISIN numbers
isin_list = corporate_bond_holdings_df['ISIN'].tolist()
# isin_list

In [ ]:
# Install and initiate driver
driver = webdriver.Chrome(ChromeDriverManager().install())

In [ ]:
# URL to test the individual sneaker page functions
my_url = 'https://cbonds.com/'

# open and maximize the browser window
driver.get(my_url)
# driver.maximize_window()

In [ ]:
# initiate the action object
action = webdriver.ActionChains(driver)

In [ ]:
# create a wait with a 5 second timeout
wait = WebDriverWait(driver, 5)

In [350]:
# create lists to append the text and links into
text_list = []
href_list = []

In [351]:
# function to grab the basic info and link to bond page from the popdown in the search bar
def scrape_correct_names(isin_list):
    for isin in isin_list:
        
        search_bar = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="mainSearch"]')))
        # click search bar
        search_bar.click()
        # random sleep time between 1 and 4 seconds
        sleep(randint(1,4))
        # type in isin number
        search_bar.send_keys(isin)
        # time sleep 5 seconds
        time.sleep(10)
        
        # grab info from isin entry pop down
        pop_down = driver.find_element_by_xpath('/html/body/div[2]/div[1]/div[1]/div[4]/div/ul[2]')
        pop_down_link = driver.find_element_by_xpath('/html/body/div[2]/div[1]/div[1]/div[4]/div/ul[2]/li/a')
        text = pop_down.text  
        href = pop_down_link.get_attribute('href')
        text_list.append(text)
        href_list.append(href)
        print(text)
        print(href)
        
        # random sleep time between 2 and 6 seconds
        sleep(randint(2,6))
        
        # select all text in the search bar and delete
        search_bar.send_keys(Keys.COMMAND, 'a')
        search_bar.send_keys(Keys.BACKSPACE)
        # random sleep time between 5 and 11 seconds
        sleep(randint(5,11))
    
    print('done')

In [ ]:
scrape_correct_names(isin_list)

In [379]:
# text_list

In [380]:
# href_list

In [355]:
# create new variables just incase they get overwritten
all_text = text_list
all_links = href_list

In [356]:
# add it to dataframe so it could be saved
scraped_isin_df = pd.DataFrame(isin_list)

In [357]:
# add the url links to scraped_isin_df as well
scraped_isin_df['urlpath'] = all_links

In [370]:
# create lists to add splitted items from the loop below
names = []
dates = []
percentages = []
currencies = []
statuses = []

In [371]:
# takes the data scraped and splits it into variables regarding the content
for text in all_text:
    new_text = text.split(", ")
    
    name = new_text[0]
    names.append(name)
    date_perct = new_text[1]
    cur_stat = new_text[2]
    
    date_perct = date_perct.split(" ")
    perct = date_perct[0]
    percentages.append(perct)
    date = date_perct[1]
    dates.append(date)
    
    cur_stat = cur_stat.split("\n")
#     print(cur_stat)
    cur = cur_stat[0]
    currencies.append(cur)
    length = len(cur_stat)
    if length == 2:
        stat = cur_stat[1]
        statuses.append(stat)
    elif length == 1:
        stat = "no_status"
        statuses.append(stat)
                              
print("done")


done


In [373]:
# adds the new variables we scraped into the dataframe
scraped_isin_df['names'] = names
scraped_isin_df['date'] = dates
scraped_isin_df['percent'] = percentages
scraped_isin_df['currency'] = currencies
scraped_isin_df['status'] = statuses

In [ ]:
# exit from the driver
driver.quit()

In [374]:
scraped_isin_df

,0,urlpath,names,date,percent,currency,status
0,BE0002239086,https://cbonds.com/bonds/178995/,Elia,27may2024,1.375%,EUR,outstanding
1,BE0002256254,https://cbonds.com/bonds/236752/,RESA SA,22jul2026,1%,EUR,outstanding
2,BE0002276450,https://cbonds.com/bonds/308139/,Elia,7apr2027,1.375%,EUR,outstanding
3,BE0002280494,https://cbonds.com/bonds/317929/,GBL,23may2024,1.375%,EUR,outstanding
4,BE0002285543,https://cbonds.com/bonds/321383/,Fluvius System Operator,23jun2025,2%,EUR,outstanding
...,...,...,...,...,...,...,...
1653,XS2299001888,https://cbonds.com/bonds/930331/,Italgas,16feb2028,0%,EUR (2556D),outstanding
1654,XS2299002423,https://cbonds.com/bonds/930335/,Italgas,16feb2033,0.5%,EUR,outstanding
1655,XS2300208928,https://cbonds.com/bonds/930909/,Snam,15aug2025,0%,EUR (1642D),outstanding
1656,XS2324772453,https://cbonds.com/bonds/958159/,Ferrovie dello Stato,25mar2028,0.375%,EUR,outstanding


In [375]:
# save the df to csv file
# scraped_isin_df.to_csv (r'/Users/gabbyvinco/Desktop/better_isin_dataset.csv', index = False, header=True)

In [ ]:
# get unique names from scraped_isin_df
unique_improved = scraped_isin_df["names"].unique()
# unique_improved

In [824]:
# replace this one because the previous name was unrecognizable to the API search
holding = unique_improved[300]
unique_improved[300] = holding.replace("Holding de Infrastructures de Transport S.A.S.","Holding d'Infrastructures de Transport S.A.S.")

In [825]:
unique_improved

array(['Elia', 'RESA SA', 'GBL', 'Fluvius System Operator', 'Befimmo',
       'Anheuser-Busch InBev', 'Brussels Airport', 'Apetra',
       'Barry Callebaut Services', 'Solvay', 'Lonza Finance Int',
       'Eni Finance', 'VGP NV', 'JAB Holdings',
       'TRATON Finance Luxembourg S.A.', 'SIX Finance (Luxembourg)',
       'Stellantis', 'BRISA-Concessao Rodoviaria', 'Energias de Portugal',
       'Metropolitano de Lisboa', 'NOS SGPS', 'Telecom Italia (TIM)',
       'Nederlandse Gasunie', 'Tennet Holding', 'Enexis Holding',
       'Alliander', 'Akzo Nobel', 'Nestle S.A.', 'Wolters Kluwer',
       'ASML Holding', 'Zapadoslovenska Energetika', 'LafargeHolcim',
       'Heineken', 'DSM', 'Royal Dutch Shell', 'Airbus',
       'SPP Distribucia', 'Citycon Oyj', 'CNH Industrial', 'Exor NV',
       'Novartis', 'Urenco', 'Eurofins Scientific', 'Unilever',
       'SPP Infrastructure', 'Redes Energeticas Nacionais (REN)',
       'Roche Holding AG', 'NN Group', 'Michelin', 'Adecco',
       'Royal Schip

In [827]:
len(unique_improved)

350

In [831]:
unique_improved = pd.DataFrame(unique_improved)

In [832]:
# save the unique_improved to csv file
# unique_improved.to_csv (r'/Users/gabbyvinco/Desktop/cbonds_list.csv', index = False, header=True)

# Using Selenium to scrape boerse-frankfurt for names

In [ ]:
# Install and initiate driver
driver = webdriver.Chrome(ChromeDriverManager().install())

In [ ]:
# URL to test the individual sneaker page functions
my_url = "https://www.boerse-frankfurt.de/bond/fr0011225143-electricite-de-france-s-a-e-d-f-4-125-12-27"

# open and maximize the browser window
driver.get(my_url)
# driver.maximize_window()

In [ ]:
# initiate the action object
action = webdriver.ActionChains(driver)

In [ ]:
# create a wait with a 5 second timeout
wait = WebDriverWait(driver, 5)

In [ ]:
# make list of ISIN numbers
isin_list = corporate_bond_holdings_df['ISIN'].tolist()
# isin_list

In [ ]:
# create lists for the function to append to
text_list = []
no_entries_list = []

In [ ]:
# function to navigate boerse-frankfurt search bar and grab basic info from popdown
def scrape_boerse_names(isin_list):
    for isin in isin_list:
        
        search_bar = wait.until(EC.element_to_be_clickable((By.XPATH, '/html/body/app-root/app-wrapper/div/div[1]/app-header/div[2]/div[2]/div/app-global-search-field-widget/div/form/mat-form-field/div/div[1]/div/input')))
        
        # click search bar
        search_bar.click()
        
        # random sleep time between 1 and 2 seconds
        sleep(randint(1,2))
        
        # type in isin number
        search_bar.send_keys(isin)
        
        # time sleep 5 seconds
        time.sleep(5)
        
        # grab info from isin entry pop down
        try:
            pop_down = driver.find_element_by_xpath('/html/body/div/div/div/div/mat-option/span/div/div[1]')
            text = pop_down.text  
            text_list.append(text)
            print(text)
        except:
            # select all text in the search bar and delete
            no_entries_list.append(isin)
            search_bar.send_keys(Keys.COMMAND, 'a')
            search_bar.send_keys(Keys.BACKSPACE)
            continue
        
        # random sleep time between 2 and 4 seconds
        sleep(randint(2,4))
        
        # select all text in the search bar and delete
        search_bar.send_keys(Keys.COMMAND, 'a')
        search_bar.send_keys(Keys.BACKSPACE)
        
        
        # random sleep time between 2 and 5 seconds
        sleep(randint(2,5))
    
    print('done')

In [ ]:
scrape_boerse_names(isin_list)

In [ ]:
# exit from the driver
driver.quit()

In [ ]:
# save list to dataframe just incase I accidentally delete/overwrite it
boerse_df = pd.DataFrame(text_list)

In [ ]:
# save the boerse_df to csv file
# boerse_df.to_csv (r'/Users/gabbyvinco/Desktop/boerse_text_list.csv', index = False, header=True)

In [ ]:
# create lists to append the split items to 
issue_year = []
maturation_year = []
name = []

In [ ]:
# split the items in the scraped list
for item in text_list:
    date = item[-5:]
    date = date.split("/")
    issue_year.append(date[0])
    maturation_year.append(date[1])
    item = item[:-5]
    name.append(item)

In [ ]:
# add the new lists to the dataframe
boerse_df["IssueYear"] = issue_year
boerse_df["MaturationYear"] = maturation_year

In [ ]:
# create new lists to append the split items to
company_name = []
interest_rate = []

In [ ]:
# split the items in the scraped list
for item in name:
    item = item.split(",")
#     print(item)
    if len(item) > 1:
        decimal = item[1]
#         print(decimal)
        first_part = item[0]
        integer = first_part[-1]
        company = first_part[:-2]
        interest = integer +"."+ decimal
        company_name.append(company)
        interest_rate.append(interest)
    if len(item) == 1 :
        not_split = item[0]
        percent = not_split[-3:]
#         print(percent)
        company = not_split[:-3]
#         print(company)
        company_name.append(company)
        interest_rate.append(integer)
    

In [ ]:
# create list to append item to
fixed_comp_name = []

In [ ]:
# remove extra space on the end if its there
for item in company_name:
    if item[-1] == " ":
        item = item[:-1]
        fixed_comp_name.append(item)
    else:
        fixed_comp_name.append(item)

In [ ]:
# fixed_comp_name

In [ ]:
# create list to append item to
comp_name = []

In [ ]:
# remove square brackets because it was giving the API search issues
for item in fixed_comp_name:
    item = item.replace(" [GBL]","")
    item = item.replace("[", "(")
    item = item.replace("]", ")")
    comp_name.append(item)

In [ ]:
# comp_name

In [ ]:
# add lists to dataframe
boerse_df["CompanyName"] = comp_name
boerse_df["InterestRate"] = interest_rate

In [ ]:
# drop unneccesary column
boerse_df = boerse_df.drop(boerse_df.columns[0], axis=1)

In [ ]:
# reorder columns
cols = ["CompanyName", "InterestRate","IssueYear","MaturationYear"]
boerse_df = boerse_df[cols]

In [ ]:
boerse_df

In [ ]:
# get unique company names
boerse_unique = boerse_df["CompanyName"]
boerse_unique = boerse_unique.unique()
boerse_unique = boerse_unique.tolist()

In [ ]:
# create a dataframe from the ISIN codes that didn't produce a result on boerse-frankfurt
missing_from_boerse = pd.DataFrame(no_entries_list)

In [ ]:
missing_from_boerse

In [ ]:
# create a list to append to and a count so that we can use it as an index to access the same
count = 0
missing_boerse_names = []

### I fixed an error here, but to check if it is right I have to run the scraper all over again because I forgot to save no_entries_list before. It takes a few hours to run and I don't want to run it again if its not absolutely neccesary.

### So everything from here on out for this section (boerse-frankfurt) is going to change. the results you see below are probably not correct

In [ ]:
# should connect the isin code we have here in the no_entries_list to the company name in the OG excel sheet

for code in no_entries_list:
    count = 0
    for isin in isin_list:
        count += 1
        if code == isin:
            name = corporate_bond_holdings_df['ISSUER'][count]
            missing_boerse_names.append(name) 

In [ ]:
missing_from_boerse["names"] = missing_boerse_names

In [ ]:
missing_names = missing_from_boerse["names"].unique()
# missing_names

In [ ]:
fixed_missing = []

In [ ]:
for item in missing_names:
    item = item.replace("√°","a")
    item = item.replace("√£", "a")
    item = item.replace("√©", "e")
    item = item.replace("Intl", "International")
    item = item.replace("Luxembg", "Luxembourg")
    item = item.replace("Luxemburg", "Luxembourg")
    fixed_missing.append(item)

In [ ]:
# fixed_missing

In [ ]:
all_companies = boerse_unique + fixed_missing
# all_companies

In [ ]:
final_boerse_list = []

In [ ]:
for item in all_companies:
    item = item.replace(".","")
    final_boerse_list.append(item)

In [ ]:
final_boerse_list = set(final_boerse_list)
len(final_boerse_list)

In [ ]:
final_boerse = pd.DataFrame(final_boerse_list)

In [ ]:
# save the unique_improved to csv file
# final_boerse.to_csv (r'/Users/gabbyvinco/Desktop/boerse_list.csv', index = False, header=True)